In [1]:
import cobra
from magine.plotting.venn_diagram_maker import create_venn3

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


ModuleNotFoundError: No module named 'magine'

In [121]:
syn = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785.xml')
syn.id = "Se"
rt = cobra.io.read_sbml_model('../models/rhodosporidium_toruloides_ifo_08804/Rt_IFO0880.xml')
rt.id = "Rt"

# Fix azotobacter model to remove the "_u"

In [122]:
def modify_compartments(model):
    for m in model.metabolites:
        if m.id[-2:] == "_u":
            m.id = m.id[:-2] # removes last two characters, namely "_u"
            m.compartment = m.id[-1] # rename compartment
    return model
av = modify_compartments(cobra.io.read_sbml_model('../models/azotobacter_vinelandii_dj/iDT1278.xml'))

av.id = "Av"

# Check resulting metabolites and compartments
print(av.metabolites)
for m in av.metabolites:
    print(m.compartment)

[<Metabolite 2pg_c at 0x7fadcaf52700>, <Metabolite 13dpg_c at 0x7fadcaf521f0>, <Metabolite prbamp_c at 0x7fadcaf52040>, <Metabolite grdp_c at 0x7fadcaf52340>, <Metabolite dms_e at 0x7fadcaf529a0>, <Metabolite fpram_c at 0x7fadcaf52880>, <Metabolite dcyt_p at 0x7fadcaf527f0>, <Metabolite fum_p at 0x7fadcaf52c40>, <Metabolite tyr__L_p at 0x7fadcaf52490>, <Metabolite feenter_e at 0x7fadcaf52760>, <Metabolite gg4abut_c at 0x7fadcaf52af0>, <Metabolite arbtn_e at 0x7fadcaf52f40>, <Metabolite aso3_c at 0x7fadcaf52310>, <Metabolite cpgn_un_e at 0x7fadcaf52e50>, <Metabolite alaala_e at 0x7fadcaf52bb0>, <Metabolite 3ohexACP_c at 0x7fadb3414190>, <Metabolite 3ocddec5eACP_c at 0x7fadcaf52fd0>, <Metabolite clpn120_p at 0x7fadcaf52c10>, <Metabolite dca_c at 0x7fadcaf52070>, <Metabolite chtbs_p at 0x7fadb3414340>, <Metabolite fusa_p at 0x7fadcaf527c0>, <Metabolite metglcur_c at 0x7fadb3414100>, <Metabolite pep_c at 0x7fadb34142e0>, <Metabolite h_c at 0x7fadcaf52e80>, <Metabolite rib__D_c at 0x7fadb34

c
p
c
c
c
c
c
c
c
c
c
c
c
c
e
p
e
p
c
c
c
c
c
c
p
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
p
p
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
e
e
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
c
e
c
c
c
c
c
c
c
e
e
e
e
c
c
c
c
c
p
c
p
p
c
c
p
p
e
c
c
c
e
p
p
e
c
e
p
c
c
e
p
p
c
e
p
e
p
c
c
e
p
e
p


In [123]:
# compare all metabolites across the three models
av_m = [i.id for i in av.metabolites]
syn_m = [i.id for i in syn.metabolites]
rt_m = [i.id for i in rt.metabolites]
#create_venn3(av_m, syn_m, rt_m, 'AV', 'SYN', 'RT')

We noticed that AV had no overlap with metabolites. The source was the use of '_u' cocompartment in the model.

In [124]:
av_all = [i.id for i in av.metabolites]
syn_all = [i.id for i in syn.metabolites]
rt_all = [i.id for i in rt.metabolites]
#create_venn3(av_all, syn_all, rt_all, 'AV', 'SYN', 'RT');

In [125]:
print(av_m[0:5])
print(av_all[0:5])

['2pg_c', '13dpg_c', 'prbamp_c', 'grdp_c', 'dms_e']
['2pg_c', '13dpg_c', 'prbamp_c', 'grdp_c', 'dms_e']


We are interesting in metabolites that are imported/exported between the organisms.

In [126]:
# comparing IDs
# compartment _e == extra_cellular
av_em = [i for i in av_m if i[-1] == 'e']
syn_em = [i for i in syn_m if i[-1] == 'e']
rt_em = [i for i in rt_m if i[-1] == 'e']
#create_venn3(av_m, syn_m, rt_m, 'AV', 'SYN', 'RT');

In [127]:
print(av_em[0:5])

['dms_e', 'feenter_e', 'arbtn_e', 'cpgn_un_e', 'alaala_e']


### Find metabolites that 'could' be missing from models

In [128]:
def check_metab_in_org(to_inspect, organism_background):
    background_core = [i.rsplit('_', 1)[0] for i in organism_background]
    found_in_org = set()
    for met in to_inspect:
        met_core = met.rsplit('_', 1)[0]
        if met_core in background_core:
            found_in_org.add(met_core)
    return found_in_org

In [129]:
check_metab_in_org(set(syn_em).difference(av_em), av_all)

{'arg__L', 'hco3', 'leu__L', 'ptrc'}

In [130]:
check_metab_in_org(set(av_em).difference(rt_em), rt_all)

{'2dglc',
 '34dhpac',
 '3hcinnm',
 '3hpp',
 '3hpppn',
 '4hoxpacd',
 '4hphac',
 'acald',
 'acgam1p',
 'acon_C',
 'acser',
 'ascb__L',
 'crn',
 'cyan',
 'cynt',
 'damp',
 'dcmp',
 'dcyt',
 'dgmp',
 'dgsn',
 'dimp',
 'dopa',
 'dump',
 'duri',
 'etoh',
 'fad',
 'fald',
 'g3pg',
 'g3ps',
 'gal1p',
 'gbbtn',
 'gdp',
 'glyald',
 'gthox',
 'gtp',
 'h2s',
 'hxan',
 'imp',
 'indole',
 'lipoate',
 'malthx',
 'maltpt',
 'maltttr',
 'mmet',
 'mobd',
 'nmn',
 'o2s',
 'octa',
 'orot',
 'pacald',
 'pheme',
 'phenol',
 'ppal',
 'progly',
 'psuri',
 'pydam',
 'pydx',
 'r5p',
 'sel',
 'skm',
 'tcynt',
 'ttdcea',
 'tungs',
 'uacgam',
 'udcpp',
 'udpacgal',
 'udpg',
 'udpgal',
 'udpglcur',
 'xmp',
 'xylu__L'}

In [131]:
check_metab_in_org(set(syn_em).difference(rt_em), rt_all)

{'cynt', 'mobd'}

# Total number of external metabolites between all three organisms

In [132]:
r = av.reactions[0]
m = av.metabolites[0]

In [133]:
all_ext_m = set(av_em).union(syn_em).union(rt_em)
print(len(all_ext_m))

484


In [151]:
# BUILD MODEL BY LOOPING THROUGH REACTIONS
# Function to creatae model for purposes of visualization with d3flux
def make_metabolite_exchange_model(org_list):
    metabolite_exchange_model = cobra.Model("Metabolite_Exchange_Model")
    # go through each organism
    for this_org in org_list:
        with this_org as org:
            # create new metabolite species (needed for d3flux)
            org_m = cobra.Metabolite(org.id)
            org_m.compartment = org.id
            
            # go through each transport reaction with an external compartment
            org_transport_er = [r for r in org.reactions if ('e' in r.compartments and len(r.compartments)>1)]
            for r in org_transport_er:
                # go through each non-external metabolite
                for m in r.metabolites:
                    # filter out exchange reaction (only allow transport reaction)
                    # AND make sure we haven't already modified the internal metabolite of interest
                    # Note: exchange reactions only contain external metabolites
                    if ((m.compartment != 'e') and (org.id not in m.compartment) ):# modify metabolite compartment and id to include organism id
                        m.compartment = m.compartment + "_" + org.id
                        m.id = m.id + "_" + org.id
                        # Add reaction between organism and metabolite
                        new_r = cobra.Reaction("Reaction_" + m.id)
                        new_r.add_metabolites({org_m:-1, m:1})
                        metabolite_exchange_model.add_reactions([new_r])
                # Change reaction id then add it to the new model
                r.id = r.id + "_" + org.id
                metabolite_exchange_model.add_reactions([r])
    return metabolite_exchange_model



In [160]:
import d3flux as d3f

syn_1 = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785.xml')
syn_1.id = "Se_1"

syn_2 = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785.xml')
syn_2.id = "Se_2"

syn_exchange_model = make_metabolite_exchange_model([syn_1, syn_2])

d3f.flux_map(syn_exchange_model)


In [157]:
syn_exchange_model.reactions[2]




Reaction identifier,O2tex_Se_Se_Se_Se_Se_Se
Name,Oxygen transport via diffusion (extracellular to periplasm)
Memory address,0x07fadb3256ee0
Stoichiometry,o2_e <=> o2_p_Se O2 O2 <=> O2 O2
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [172]:
# BUILD MODEL BY LOOPING THROUGH EXTERNAL METABOLITES ONLY

# Function to create model for purposes of visualization with d3flux
def make_metabolite_exchange_model_only_shared_external_metabolites(org_list):
    metabolite_exchange_model = cobra.Model("Metabolite_Exchange_Model")
    # go through each organism
    for this_org in org_list:
        with this_org as org:
            # create new metabolite species (needed for d3flux)
            org_m = cobra.Metabolite(org.id)
            org_m.compartment = org.id
            
            # go through each non-external metabolite
            org_m_reactions = []
            org_em = [m for m in org.metabolites if m.compartment == 'e']
            for m in org_em:
                # Add new reaction for this metabolite
                new_r = cobra.Reaction("Reaction_" + m.id + "_" + org.id)
                new_r.add_metabolites({org_m:-1, m:1})
                org_m_reactions.append(new_r)

            # Add new reactions to new model
            metabolite_exchange_model.add_reactions(org_m_reactions)
    
    lone_metabolites = [m for m in metabolite_exchange_model.metabolites if len(m.reactions)==1]
    metabolite_exchange_model.remove_metabolites(lone_metabolites, destructive=True)

    return metabolite_exchange_model


In [173]:

syn = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785.xml')
syn.id = "Se_Se"

av = modify_compartments(cobra.io.read_sbml_model('../models/azotobacter_vinelandii_dj/iDT1278.xml'))
av.id = "Av_Av"

rt = cobra.io.read_sbml_model('../models/rhodosporidium_toruloides_ifo_08804/Rt_IFO0880.xml')
rt.id = "Rt_Rt"

av_syn_rt_exchange_model = make_metabolite_exchange_model_only_shared_external_metabolites([av, syn, rt])

d3f.flux_map(av_syn_rt_exchange_model)
